In [2]:
import freud
import gsd.fl
import gsd.hoomd
import numpy as np
#import msd_plot as msd
import matplotlib.pyplot as plt
import signac

In [3]:
def find_starting_frame(trajectory):
    
    box_dims = trajectory[-1].configuration.box[:3]
    for i in range(0, len(trajectory)):
        if np.array_equal(trajectory[i].configuration.box[:3], box_dims):
            starting_index = i
            break
    return(starting_index)

def plot_rdf(files, labels, types, title, num_frames=1, r_max=8, dr=0.25):
    
    fig = plt.figure()
    for i, f in enumerate(files):
        rdf, x, y = average_rdf(f, types, num_frames, r_max, dr)
        plt.plot(x, y, label = labels[i])
        rdf.reset()
    plt.legend()
    plt.show()
    plt.title(title)
    rdf.reset()

def average_rdf(file, types, num_frames=1, r_max=8, dr=0.25):
    
    print('FILE = {}'.format(file))
    gsd_file = gsd.fl.GSDFile(file, "rb")
    trajectory = gsd.hoomd.HOOMDTrajectory(gsd_file)
    starting_frame = find_starting_frame(trajectory) # First frame at final box dims
    print("STARTING FRAME = {}".format(starting_frame))
    if num_frames > len(trajectory) - starting_frame:
        print('Using this many frames will result in multiple box sizes being used')
        return
    x, y, z = trajectory[-1].configuration.box[:3]
    if r_max == None:
        if x == y and x == z:
            r_max = x // 2
        else:
            r_max = 8
    system_box = freud.box.Box(x, y, z)
    rdf = freud.density.RDF(r_max, dr)
    for i in range(1, num_frames + 1):
        frame = trajectory[-i]
        rdf = single_frame_rdf(frame=frame, types=types, rdf=rdf, box=system_box)
    x_data = rdf.R
    y_data = rdf.RDF
    return rdf, x_data, y_data
    
def single_frame_rdf(frame, types, rdf, box):
    
    if types[0] == 'all':
        positions = frame.particles.position
    else:
        positions = []
        #for atom_type in types:
        for idx, type_id in enumerate(frame.particles.typeid):
            if frame.particles.types[type_id] in types:
                positions.append(frame.particles.position[idx])
    print('# OF PARTICLES = {}'.format(len(positions)))
    rdf.accumulate(box=box, ref_points = positions)
    return rdf

In [29]:
# Set up directories:

anneal_5mer_path = 'annealed-systems/5mer'
anneal_5mer_ws = 'annealed-systems/5mer/7aa807f49398ab18289ddbeb45e2c53b/trajectory.gsd'
anneal_10mer_path = 'annealed-systems/10mer'
anneal_10mer_ws = 'annealed-systems/10mer/52a8fd463d0e6d9a4cdc758e9991a412/trajectory.gsd'
anneal_15mer_path = 'annealed-systems/15mer'
anneal_15mer_ws = 'annealed-systems/15mer/25ce07344215f94145d689abb811c799/trajectory.gsd'
quench_5mer_path = 'quenched-systems/5mer'
quench_5mer_ws = 'quenched-systems/5mer/7aa807f49398ab18289ddbeb45e2c53b/trajectory.gsd'
quench_10mer_path = 'quenched-systems/10mer'
quench_10mer_ws = 'quenched-systems/10mer/2b6eac3d791efc8fcf2aa79a66e9d95c/trajectory.gsd'
quench_15mer_path = 'quenched-systems/15mer'
quench_15mer_ws = 'quenched-systems/15mer/ba4e6c861394a92f5374d85990e6ba28/trajectory.gsd'
test_path = 'hot-lowD-test.gsd'

In [54]:
plot_rdf(files=[anneal_5mer_ws, quench_5mer_ws],
         labels=['annealed', 'quenched'],
         types=['o'],
         num_frames=5,
         title = 'End Groups')

FILE = annealed-systems/5mer/7aa807f49398ab18289ddbeb45e2c53b/trajectory.gsd
STARTING FRAME = 4
# OF PARTICLES = 1000
# OF PARTICLES = 1000
# OF PARTICLES = 1000
# OF PARTICLES = 1000
# OF PARTICLES = 1000
FILE = quenched-systems/5mer/7aa807f49398ab18289ddbeb45e2c53b/trajectory.gsd
STARTING FRAME = 4
# OF PARTICLES = 1000
# OF PARTICLES = 1000
# OF PARTICLES = 1000
# OF PARTICLES = 1000
# OF PARTICLES = 1000


In [4]:
project = signac.get_project(search=False)
project

,sp.molecule,sp.n_compounds,sp.density,sp.e_factor,sp.kT_reduced,sp.tau,sp.n_steps,sp.dt,sp.remove_hydrogens,sp.original_id,...,doc.mass,doc.mass_units,doc.energy,doc.energy_units,doc.distance,doc.distance_units,doc.T_SI,doc.T_unit,doc.real_timestep,doc.time_unit
fad8a07a07787fe67137f98bc8b2f9a6,PTB7_5mer_smiles,200,1.20,0.5,1.4,1,30000000.0,0.001,True,7aa807f49398ab18289ddbeb45e2c53b,...,32.06,amu,1.046,kj/mol,0.356359,nm,176.0,K,1.973,fs
6c9837bf78ad03b2ce9f15460cfd9aa5,PTB7_5mer_smiles,200,1.20,0.5,1.4,1,30000000.0,0.001,True,7aa807f49398ab18289ddbeb45e2c53b,...,32.06,amu,1.046,kj/mol,0.356359,nm,176.0,K,1.973,fs
d6a89e9ce106ab76844cfe4038299129,PTB7_5mer_smiles,200,0.95,0.5,2.0,1,30000000.0,0.001,True,29d7946b2ba627edd3db25f54bb2cda9,...,32.06,amu,1.046,kj/mol,0.356359,nm,252.0,K,1.973,fs
037b5f2cc6ef8471acc32295aed1a4e3,PTB7_5mer_smiles,200,0.95,0.5,2.0,1,20000000.0,0.001,True,b057e798fd4e569cbc5924e768b2dcfb,...,32.06,amu,1.046,kj/mol,0.356359,nm,252.0,K,1.973,fs
af18150bfdab3c05f2f3f2b2ade82558,PTB7_5mer_smiles,200,1.10,0.5,1.7,1,30000000.0,0.001,True,c6b746b44bac29f436c3c33158624dc1,...,32.06,amu,1.046,kj/mol,0.356359,nm,214.0,K,1.973,fs
c33d16ecbf3fc9a1c244c23880e02c42,PTB7_5mer_smiles,200,1.10,0.5,1.7,1,30000000.0,0.001,True,c6b746b44bac29f436c3c33158624dc1,...,32.06,amu,1.046,kj/mol,0.356359,nm,214.0,K,1.973,fs


In [10]:
# Compare Average Sq for each state-point:
%matplotlib qt

features = ['side-chains', 'backbone', 'mono-ends']
feature_groups = [['c3'], ['cc', 'cd', 'ca', 'ss'], ['f']]
job_list = []
for job in project:
    if job.sp['kT_reduced'] == 2.0:
        if job.sp['process'] == 'quench':
            job_list.append(job)
            print(job.get_id())
        if job.sp['process'] == 'anneal':
            job_list.append(job)
            print(job.get_id())
    else:
        pass

#for j in job_list:
#    work_dir = j.workspace()
for index, group in enumerate(feature_groups):
    files = [j.workspace()+'/trajectory.gsd' for j in job_list]
    plot_rdf(files=files, labels=['quench', 'anneal'], title='High T-Low D {}'.format(features[index]), 
             types=group, num_frames=3, dr=0.10)

d6a89e9ce106ab76844cfe4038299129
037b5f2cc6ef8471acc32295aed1a4e3
FILE = /home/chris/cme/opv-updates/meetings/dec-2-19/workspace/d6a89e9ce106ab76844cfe4038299129/trajectory.gsd
STARTING FRAME = 4
# OF PARTICLES = 24000
# OF PARTICLES = 24000
# OF PARTICLES = 24000
FILE = /home/chris/cme/opv-updates/meetings/dec-2-19/workspace/037b5f2cc6ef8471acc32295aed1a4e3/trajectory.gsd
STARTING FRAME = 5
# OF PARTICLES = 24000
# OF PARTICLES = 24000
# OF PARTICLES = 24000
FILE = /home/chris/cme/opv-updates/meetings/dec-2-19/workspace/d6a89e9ce106ab76844cfe4038299129/trajectory.gsd
STARTING FRAME = 4
# OF PARTICLES = 20000
# OF PARTICLES = 20000
# OF PARTICLES = 20000
FILE = /home/chris/cme/opv-updates/meetings/dec-2-19/workspace/037b5f2cc6ef8471acc32295aed1a4e3/trajectory.gsd
STARTING FRAME = 5
# OF PARTICLES = 20000
# OF PARTICLES = 20000
# OF PARTICLES = 20000
FILE = /home/chris/cme/opv-updates/meetings/dec-2-19/workspace/d6a89e9ce106ab76844cfe4038299129/trajectory.gsd
STARTING FRAME = 4
# OF PAR

In [ ]:
'def plot_rdf(files, labels, types, title, num_frames=1, r_max=8, dr=0.25):'

In [18]:
%matplotlib qt
fig = plt.figure()
plt.plot(x0, y0, label='Quenched')
plt.plot(x1, y1, label='Annealed')
plt.plot(x2, y2, label='Test')
plt.legend()
plt.title('RDF of PTB7 Features - 5mer')
plt.show()